Face ID Based Smart Door Bell with Secured Door Lock  

In [ ]:
import cv2
import cvzone 
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.SerialModule import SerialObject
from cvzone.FPS import FPS
from cvzone import cornerRect, putTextRect
import pandas as pd
from datetime import datetime
import time
import os
import requests


TOKEN = '7230981972:AAEFo2FxODCeX03-a0aqH33Ka8xSZwcum3I'
CHAT_ID = '5029073982'



fpsReader = FPS(avgCount=30)

# Initialize the serial object for Arduino communication
arduino = SerialObject("COM7", digits=1)

# Initialize the face detector
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30) #30


detector = FaceDetector(0.7)

# Define the directory where you want to save the Excel file
save_directory = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Door Bell Images'


#Initialize the Excel file

excel_file = 'doorbell_log.xlsx'

try:
    df = pd.read_excel(excel_file)
except FileNotFoundError:
    df = pd.DataFrame(columns=['No.', 'Date', 'Time','Door Lock'])

# Initialize the doorbell counter
doorbell_count = len(df) + 1

# Time when the face was first detected
face_detected_time = None

# Flag to check if a face has been logged
face_logged = False

flag_lock = True

master_flag = False
last_flag = False 
# Initialize the last send time variable
last_send_time = 0

while True:

    flag=arduino.getData()
    try: 
        if flag[0]=='7':
            last_flag = True
            #flag_lock = True
            print(flag[0])
    except:
     pass

    success, img = cap.read()
    H,W,_ = img.shape
    roix1, roiy1, roix2, roiy2 = int(W*0.35), int(H*0.05), int(W-0.35*W), int(H-0.4*H)
    imgroi = img[roiy1:roiy2, roix1:roix2]
    cornerRect(img, [roix1, roiy1, roix2-roix1, roiy2-roiy1],rt=2,colorR=(0,0,0),colorC=(255,0,0))
    imgroi, bbox = detector.findFaces(imgroi)
    

    if bbox:
        
        #arduino.sendData([1])

        putTextRect(img,'CHECK DOOR',(roix1+20, roiy2+50), scale=1.5, colorR= (0,0,210), colorB = (0,0,0), offset=8, thickness=2, border=3)
        current_time = time.time()
        # Check if 1 second has passed since the last send
        if current_time - last_send_time >= 2: #1.5 
            arduino.sendData([1])
            last_send_time = current_time  # Update the last send time

        if face_detected_time is None:
            # Start the timer when a face is first detected
            face_detected_time = time.time()
        else:
            # Check if 2 seconds have passed since the face was first detected
            if time.time() - face_detected_time >= 2 and not face_logged:
                
                #flag=arduino.getData()
                # Save the date and time of the doorbell ring
                current_time = datetime.now()
                date_str = current_time.strftime("%Y-%m-%d")
                time_str = current_time.strftime("%H:%M:%S")
                new_entry = pd.DataFrame({'No.': [doorbell_count], 'Date': [date_str], 'Time': [time_str], 'Door Lock': ["Locked"]})
                df = pd.concat([df, new_entry], ignore_index=True)
                df.to_excel(excel_file, index=False)
                doorbell_count += 1

                # Save the cropped face image
                #img_filename = os.path.join(save_directory, f'{doorbell_count - 1}.jpg')
                img_filename = os.path.join(save_directory, f'{doorbell_count-1 }.jpg')
                imgWrite = cv2.resize(img,(int(0.5*W), int(0.5*H)))
                cv2.imwrite(img_filename, imgWrite)

                #Send the image to telegram

                msg=str(doorbell_count-1)
                location = img_filename
                files = {'photo':open(location,'rb')}

                url = f"https://api.telegram.org/bot{TOKEN}/sendPhoto?chat_id={CHAT_ID}&caption={msg}"
                r = requests.post(url,files=files)
                #print(r.json())


                # Set the face_logged flag to True
                face_logged = True
                flag_lock = True 
                master_flag = True

            #flag=arduino.getData()

            if master_flag: 

                       
                #if flag[0] == '8' and  flag_lock:
                    
                 if last_flag:
                    print("Last Flag")
                    #print(flag[0])
                    # Save the date and time of the doorbell ring

                    current_time = datetime.now()
                    date_str = current_time.strftime("%Y-%m-%d")
                    time_str = current_time.strftime("%H:%M:%S")
                    new_entry = pd.DataFrame({'No.': [doorbell_count-1], 'Date': [date_str], 'Time': [time_str], 'Door Lock': ["Unlocked"]})
                    
                    df.iloc[-1] = new_entry.iloc[0]
                    df.to_excel(excel_file, index=False)

                    #df = pd.concat([df, new_entry], ignore_index=True)
                    #df.to_excel(excel_file, index=False)
                    #doorbell_count += 1
                    flag_lock = False
                    master_flag = False 
                    last_flag = False 
                   
                    
                    


    else:
        #arduino.sendData([0])
        # Reset the timer and face_logged flag
        face_detected_time = None
        face_logged = False
        master_flag = False
        putTextRect(img,'SCAN YOUR FACE',(roix1-10, roiy2+50), scale=1.5, colorR= (0,200,0), colorB = (0,0,0), offset=8, thickness=2, border=3)

        if last_flag and flag_lock: 
                    
                    #print(flag[0])
                    # Save the date and time of the doorbell ring
                    current_time = datetime.now()
                    date_str = current_time.strftime("%Y-%m-%d")
                    time_str = current_time.strftime("%H:%M:%S")
                    new_entry = pd.DataFrame({'No.': [doorbell_count-1], 'Date': [date_str], 'Time': [time_str], 'Door Lock': ["Unlocked"]})
                    
                    df.iloc[-1] = new_entry.iloc[0]
                    df.to_excel(excel_file, index=False)
                    print("Last Flag Bottom")
                    #df = pd.concat([df, new_entry], ignore_index=True)
                    #df.to_excel(excel_file, index=False)
                    #doorbell_count += 1
                    flag_lock = False
                    master_flag = False 
                    last_flag = False

        last_flag = False
    
    
    # fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(20, 50),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    
    cv2.imshow("Face Detector", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
